## What's this TensorFlow business?

You've written a lot of code in this assignment to provide a whole host of neural network functionality. Dropout, Batch Norm, and 2D convolutions are some of the workhorses of deep learning in computer vision. You've also worked hard to make your code efficient and vectorized.

For the last part of this assignment, though, we're going to leave behind your beautiful codebase and instead migrate to one of two popular deep learning frameworks: in this instance, TensorFlow (or PyTorch, if you switch over to that notebook)

#### What is it?
TensorFlow is a system for executing computational graphs over Tensor objects, with native support for performing backpropogation for its Variables. In it, we work with Tensors which are n-dimensional arrays analogous to the numpy ndarray.

#### Why?

* Our code will now run on GPUs! Much faster training. Writing your own modules to run on GPUs is beyond the scope of this class, unfortunately.
* We want you to be ready to use one of these frameworks for your project so you can experiment more efficiently than if you were writing every feature you want to use by hand. 
* We want you to stand on the shoulders of giants! TensorFlow and PyTorch are both excellent frameworks that will make your lives a lot easier, and now that you understand their guts, you are free to use them :) 
* We want you to be exposed to the sort of deep learning code you might run into in academia or industry. 

## How will I learn TensorFlow?

TensorFlow has many excellent tutorials available, including those from [Google themselves](https://www.tensorflow.org/get_started/get_started).

Otherwise, this notebook will walk you through much of what you need to do to train models in TensorFlow. See the end of the notebook for some links to helpful tutorials if you want to learn more or need further clarification on topics that aren't fully explained here.

## Load Datasets


In [5]:
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import pandas as pd

In [6]:
X_train = pd.read_csv('./yahoo_answers_csv/train.csv', header = None, nrows = 100)

In [7]:
print(X_train)

     0                                                  1  \
0    5  why doesn't an optical mouse work on a glass t...   
1    6       What is the best off-road motorcycle trail ?   
2    3             What is Trans Fat? How to reduce that?   
3    7                         How many planes Fedex has?   
4    7  In the san francisco bay area, does it make se...   
5    5           What's the best way to clean a keyboard?   
6    2     Why do people blush when they are embarrassed?   
7    8  Is Lin Qingxia (aka Brigitte Lin) "the most be...   
8    5                    What is the origin of "foobar"?   
9    2                     How the human species evolved?   
10   4  Who said the statement below and what does it ...   
11   4                How do I find an out of print book?   
12   7  What are some tips on finding a good mortgage ...   
13   5  what's the best way to create a bootable windo...   
14   9  what is the reason for the increasing divorce ...   
15   2                  

In [ ]:
# clear old variables
tf.reset_default_graph()

# setup input (e.g. the data that changes every batch)
# The first dim is None, and gets sets automatically based on batch size fed in
L = 1024 # 1014, 1024 for large 256 for small
NOUT = 2048 # 2048 for large, 1024 for small 
X = tf.placeholder(tf.float32, [None, 70, L])
y = tf.placeholder(tf.int64, [None])
is_training = tf.placeholder(tf.bool)

def simple_model(X,y):    
    conv1 = tf.contrib.layers.conv1d(X, filters=1, kernel_size=7, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv1") 
    pool1 = tf.nn.max_pool(value=conv1, pool_size=3, strides=1, padding='VALID', data_format='NHWC', scope="pool1") 
    
    conv2 = tf.contrib.layers.conv1d(pool1, filters=1, kernel_size=7, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv2") 
    pool2 = tf.nn.max_pool(value=conv2, pool_size=3, strides=1, padding='VALID', data_format='NHWC', scope="pool2")
    
    conv3 = tf.contrib.layers.conv1d(pool2, filters=1, kernel_size=3, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv3") 
    
    conv4 = tf.contrib.layers.conv1d(conv3, filters=1, kernel_size=3, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv4") 
    
    conv5 = tf.contrib.layers.conv1d(conv4, filters=1, kernel_size=3, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv5") 
    
    conv6 = tf.contrib.layers.conv1d(conv5, filters=1, kernel_size=3, stride=1, padding='SAME', 
                                   activation_fn=tf.nn.relu, reuse=None, scope="conv6") 
    pool6 = tf.nn.max_pool(value=conv6, pool_size=3, strides=1, padding='VALID', data_format='NHWC', scope="pool6")
    
    fc7 = tf.contrib.layers.fully_connected(inputs=tf.contrib.layers.flatten(pool6), num_outputs=NOUT, reuse=None, 
                                      scope="fc7")
    fc8 = tf.contrib.layers.fully_connected(inputs=tf.contrib.layers.flatten(fc7), num_outputs=NOUT, reuse=None, 
                                      scope="fc8")
    
    return fc8

y_out = simple_model(X,y)

# define our loss
total_loss = tf.losses.hinge_loss(tf.one_hot(y,10),logits=y_out)
mean_loss = tf.reduce_mean(total_loss)

# define our optimizer
optimizer = tf.train.AdamOptimizer(5e-4) # select optimizer and set learning rate
train_step = optimizer.minimize(mean_loss)